In [2]:

import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from tqdm.auto import tqdm
from glob import glob
from shapely.geometry import LineString, Point
import folium
pd.set_option('display.max_columns', None)

In [19]:
new_shoreline = gpd.GeoDataFrame.from_features({"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[295.177354,32.250787],[295.177622,32.25134],[295.178175,32.251825],[295.178894,32.25227],[295.179468,32.252456],[295.179961,32.252492]]}}]}, crs="EPSG:4326")
new_shoreline

,geometry
0,"LINESTRING (295.17735 32.25079, 295.17762 32.2..."


In [20]:
new_shoreline.explore(tiles="ESRI.WorldImagery")

In [21]:
shorelines = gpd.read_file("shorelines.geojson")
transects = gpd.read_file("transects_extended.geojson")
poly = gpd.read_file("polygons.geojson")

In [22]:
new_siteid = "ber0002"
new_shoreline["id"] = new_siteid
new_shoreline

,geometry,id
0,"LINESTRING (295.17735 32.25079, 295.17762 32.2...",ber0002


In [23]:
pd.concat((shorelines, new_shoreline)).to_file("shorelines.geojson", driver="GeoJSON")

In [24]:
new_poly = gpd.GeoDataFrame([{"id": new_siteid}], geometry=new_shoreline.to_crs(3769).buffer(100).minimum_rotated_rectangle().to_crs(4326), crs=4326)
new_poly.explore()

In [25]:
pd.concat((poly, new_poly)).to_file("polygons.geojson", driver="GeoJSON")

In [30]:
def create_transects(line, spacing=25, transect_length=300):
    transects = []
    distances = np.arange(0, line.length, spacing)

    for distance in distances:
        # Find point along the average line
        point = line.interpolate(distance)
        
        # Find the direction of the line at this point (tangent direction)
        nearest_point_ahead = line.interpolate(min(distance + 1e-6, line.length))
        direction = np.arctan2(nearest_point_ahead.y - point.y, nearest_point_ahead.x - point.x)
        
        # Rotate 90 degrees (perpendicular) and extend to create a transect
        transect = LineString(reversed([
            Point(
                point.x - transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y - transect_length / 2 * np.sin(direction + np.pi / 2)
            ),
            Point(
                point.x + transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y + transect_length / 2 * np.sin(direction + np.pi / 2)
            )
        ]))
        
        transects.append(transect)
    
    transects = gpd.GeoDataFrame(geometry=transects, crs=3769).to_crs(4326)
    return transects

new_transects = create_transects(new_shoreline.to_crs(3769).geometry.iloc[0])
display(new_transects)
m = new_transects.explore(tiles="Esri.WorldImagery")
new_shoreline.explore(m=m)
new_poly.boundary.explore(m=m)
gpd.GeoSeries(new_transects.geometry.apply(lambda line: Point(line.coords[0])), crs=new_transects.crs).explore(m=m, color="red", name="transect start")
print("Make sure the origin is inland")
m

,geometry
0,"LINESTRING (-64.82412 32.2513, -64.82117 32.25..."
1,"LINESTRING (-64.82402 32.25151, -64.82107 32.2..."
2,"LINESTRING (-64.82392 32.25172, -64.82097 32.2..."
3,"LINESTRING (-64.82346 32.25234, -64.82117 32.2..."
4,"LINESTRING (-64.82327 32.2525, -64.82099 32.25..."
5,"LINESTRING (-64.82309 32.25266, -64.8208 32.25..."
6,"LINESTRING (-64.82269 32.25297, -64.82081 32.2..."
7,"LINESTRING (-64.82247 32.2531, -64.8206 32.25091)"
8,"LINESTRING (-64.82226 32.25323, -64.82038 32.2..."
9,"LINESTRING (-64.82204 32.25336, -64.82017 32.2..."


Make sure the origin is inland


In [31]:
new_transects["id"] = new_siteid + "-" + new_transects.index.astype(str).str.pad(4, fillchar="0")
new_transects["site_id"] = new_siteid
#new_transects["beach_slope"] = .1
new_transects

,geometry,id,site_id
0,"LINESTRING (-64.82412 32.2513, -64.82117 32.25...",ber0002-0000,ber0002
1,"LINESTRING (-64.82402 32.25151, -64.82107 32.2...",ber0002-0001,ber0002
2,"LINESTRING (-64.82392 32.25172, -64.82097 32.2...",ber0002-0002,ber0002
3,"LINESTRING (-64.82346 32.25234, -64.82117 32.2...",ber0002-0003,ber0002
4,"LINESTRING (-64.82327 32.2525, -64.82099 32.25...",ber0002-0004,ber0002
5,"LINESTRING (-64.82309 32.25266, -64.8208 32.25...",ber0002-0005,ber0002
6,"LINESTRING (-64.82269 32.25297, -64.82081 32.2...",ber0002-0006,ber0002
7,"LINESTRING (-64.82247 32.2531, -64.8206 32.25091)",ber0002-0007,ber0002
8,"LINESTRING (-64.82226 32.25323, -64.82038 32.2...",ber0002-0008,ber0002
9,"LINESTRING (-64.82204 32.25336, -64.82017 32.2...",ber0002-0009,ber0002


In [33]:
pd.concat((transects, new_transects)).to_file("transects_extended.geojson", driver="GeoJSON")